In [1]:
from keras.models import Model
from keras.layers import Input, Dense, \
    RevCompConv1D, Conv2D, Flatten, Dropout, Merge, \
    Activation, MaxPooling1D, RevCompConv1DBatchNorm, \
    BatchNormalization, DenseAfterRevcompConv1D
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import keras

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt

import pickle
import numpy as np
import sys
import os
sys.path.append('../../utils/')
import input_data_config as input_data
from callbacks import BatchHistory

import tensorflow as tf

from joblib import Parallel, delayed
import yaml

from ggplot import *

Using TensorFlow backend.
/home/tshimko/.virtualenvs/DeepRegret/lib/python3.5/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/tshimko/.virtualenvs/DeepRegret/lib/python3.5/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/tshimko/.virtualenvs/DeepRegret/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
fig_dir = 'figures/'
log_dir = 'logs/'
out_dir = 'processed_data/'

if not os.path.exists(fig_dir): os.mkdir(fig_dir)
if not os.path.exists(log_dir): os.mkdir(log_dir)
if not os.path.exists(out_dir): os.mkdir(out_dir)
if not os.path.exists('model_configs/'): os.mkdir('model_configs/')

In [3]:
def test_models(gpu,
                model_num,
                units=[32, 64, 128, 256],
                filters=[128, 256, 512, 1024],
                kernel_widths=[3, 5, 7, 9],
                reg_layers=[1, 2, 3, 4],
                seq_layers=[1, 2, 3, 4],
                concat_layers=[1, 2, 3, 4]):

    num_reg = 472
    seq_length = 1000

    reg_layers = int(np.random.choice(reg_layers, 1)[0])
    reg_units = np.random.choice(units, reg_layers).tolist()
    
    seq_layers = int(np.random.choice(seq_layers, 1)[0])
    filters = sorted(np.random.choice(filters, seq_layers).tolist(),
                   reverse=True)
    kernels = sorted(np.random.choice(kernel_widths, seq_layers).tolist())
    
    concat_layers = int(np.random.choice(concat_layers, 1)[0])
    concat_units = np.random.choice(units, concat_layers).tolist()
    
    config = {
        'model_num': model_num,
        'num_reg': num_reg,
        'seq_length': seq_length,
        'reg_layers': reg_layers,
        'reg_units': reg_units,
        'seq_layers': seq_layers,
        'filters': filters,
        'kernels': kernels,
        'concat_layers': concat_layers,
        'concat_units': concat_units
    }
    
    yaml.safe_dump(config,
                   open('model_configs/model' + str(model_num) + '.yaml', 'w'))

    with tf.device(gpu):
    
        # Build regression model
        print('INFO - %s'%('building regression model.'))
        reg_input = Input(shape=(num_reg,),dtype='float32',name='reg_input')
        x = reg_input
        n = 0

        while reg_layers > 1:
            x = Dense(reg_units[n], activation='relu')(x)

            n += 1
            reg_layers -= 1

        reg_output = Dense(reg_units[n], activation='relu')(x)

        # Build sequence model
        units = 512
        seq_input=Input(shape=(seq_length,4),dtype='float32',name='seq_input')
        x=seq_input
        
        n = 0
        
        while seq_layers > 0:
            x=RevCompConv1D(nb_filter=filters[n],filter_length=kernels[n],border_mode='valid')(x)
            x=RevCompConv1DBatchNorm()(x)
            x=Activation('relu')(x)
            x=MaxPooling1D(pool_length=4,border_mode='valid')(x)
            
            n += 1
            seq_layers -= 1
            
        seq_output=DenseAfterRevcompConv1D(output_dim=units)(x)

        # Build concatenate model
        print('INFO - %s'%('building concatenate model.'))
        
        x = Merge(mode='concat',concat_axis=1)([reg_output,seq_output])
        n = 0

        while concat_layers > 0:
            x = Dense(concat_units[n], activation='relu')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            x = Dropout(0.5, seed=42)(x)

            n += 1
            concat_layers -= 1
            
        rgs_output = Dense(1, activation='linear',name='rgs_output')(x)

        # Compile and train the model
        model = Model(output=[rgs_output], input=[reg_input, seq_input])
        model.compile(loss={'rgs_output':'mean_squared_error'},optimizer='sgd')
        
        print('INFO - %s'%('loading data.'))
        train,val,test=input_data.read_data_sets(train_pct=80,
                                                 val_pct=10,
                                                 test_pct=10,
                                                 mode='random',
                                                 seq_file='../../data/yeast_promoters.txt',
                                                 expr_file='../../data/complete_dataset.txt',
                                                 reg_names_file='../../data/reg_names_R.txt',
                                                 conv1d=True)
        
        print('INFO - %s'%('training model.'))
        reduce_lr=ReduceLROnPlateau(verbose=1,factor=0.5, patience=5)
        early_stopping=EarlyStopping(monitor='val_loss',patience=10)
        checkpoint=ModelCheckpoint(filepath="%s/model{model_num:03d}.{epoch:02d}-{val_loss:.4f}.hdf5"%(log_dir), monitor='val_loss')
        history=model.fit({'seq_input':train['seq'],'reg_input':train['reg']},{'rgs_output':train['expr']},
            validation_data=({'seq_input':val['seq'],'reg_input':val['reg']},{'rgs_output':val['expr']}),
            nb_epoch=200,
            batch_size=100,
            callbacks=[early_stopping,checkpoint,reduce_lr],
            verbose=1)

        with open('%s/model{model_num:03d}_history.pkl'%(log_dir),'wb') as f:
            pickle.dump([history.history],f)
        with open('%s/model{model_num:03d}_history.pkl'%(log_dir),'rb') as f:
            x=pickle.load(f)


        # Plot the learning curve:
        history=pd.DataFrame(x[0])
        history['epoch']=(range(1,history.shape[0]+1))
        history_melt=pd.melt(history,id_vars=['epoch'],value_vars=['loss','val_loss'],var_name='type',value_name='loss')

        p1=ggplot(history_melt,aes('epoch','loss',color='type'))+geom_line()+theme_bw()
        p1.save(filename='%s/model{model_num:03d}_learning_curve.png'%(fig_dir))


        # Plot prediction vs ground truth: 
        pred=model.predict({'seq_input':test['seq'],'reg_input':test['reg']},batch_size=100,verbose=1)
        plt.figure()
        plt.scatter(pred,test['expr'])
        plt.savefig("%s/model{model_num:03d}_pred_vs_obs.png"%(fig_dir))
        output=np.column_stack((test['expr'], pred[:,0]))
        np.savetxt("%s/model{model_num:03d}_prediction.txt"%(out_dir), output,delimiter='\t')
        
        model_num += 1
        

In [4]:
test_models('/gpu:0', 1)

INFO - building regression model.
INFO - building concatenate model.
INFO - loading data.
INFO - training model.
Train on 845208 samples, validate on 105651 samples
Epoch 1/200
 49900/845208 [>.............................] - ETA: 601s - loss: 0.7550

KeyboardInterrupt: 